In [173]:
import pandas as pd
from datetime import datetime
import requests

In [174]:
start_date = '2023-01-01'
end_date = '2024-05-27'

In [175]:
url = f'https://api.energy-charts.info/price?bzn=SI&start={start_date}&end={end_date}'
response = requests.get(url)

data = response.json()

In [176]:
print('Number of price items:', len(data['price']))
print('Number of unix time items:', len(data['unix_seconds']))

if len(data['price']) == len(data['unix_seconds']):
    print('Lengths are matching the expected values')

Number of price items: 12311
Number of unix time items: 12311
Lengths are matching the expected values


In [177]:
print('Response data from:', datetime.fromtimestamp(data['unix_seconds'][0]).strftime('%Y-%m-%d %H:%M:%S'))
print('Response data to:', datetime.fromtimestamp(data['unix_seconds'][-1]).strftime('%Y-%m-%d %H:%M:%S'))

Response data from: 2023-01-01 00:00:00
Response data to: 2024-05-27 23:00:00


In [178]:
daily_price_data = []
for i in range(len(data['price'])):
    timestamp = datetime.fromtimestamp(data['unix_seconds'][i]).strftime('%Y-%m-%dT%H:%M:%S')
    price = data['price'][i]
    
    new_data = {'date': timestamp, 'price': price}
    daily_price_data.append(new_data)

In [179]:
df = pd.DataFrame(daily_price_data)
df.head(48)

,date,price
0,2023-01-01T00:00:00,13.31
1,2023-01-01T01:00:00,-0.07
2,2023-01-01T02:00:00,-0.47
3,2023-01-01T03:00:00,-1.56
4,2023-01-01T04:00:00,-1.94
5,2023-01-01T05:00:00,-1.42
6,2023-01-01T06:00:00,-2.97
7,2023-01-01T07:00:00,-0.12
8,2023-01-01T08:00:00,-0.06
9,2023-01-01T09:00:00,0.47


In [180]:
weather_url = f'https://archive-api.open-meteo.com/v1/archive?latitude=46.0833&longitude=15&start_date={start_date}&end_date={end_date}&hourly=temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,cloud_cover,wind_speed_10m&timezone=Europe%2FBerlin'
response = requests.get(weather_url)

weather_data = response.json()

In [181]:
daily_weather_data = []
for i in range(len(weather_data['hourly']['time'])):
    temperature = weather_data['hourly']['temperature_2m'][i]
    humidity = weather_data['hourly']['relative_humidity_2m'][i]
    precipitation = weather_data['hourly']['precipitation'][i]
    cloud_cover = weather_data['hourly']['cloud_cover'][i]
    wind_speed = weather_data['hourly']['wind_speed_10m'][i]
    
    new_data = {'date': weather_data['hourly']['time'][i], 'temperature': temperature, 'humidity': humidity, 'precipitation': precipitation, 'cloud_cover': cloud_cover, 'wind_speed': wind_speed}
    daily_weather_data.append(new_data)

In [182]:
df_weather = pd.DataFrame(daily_weather_data)
df_weather.head(48)

,date,temperature,humidity,precipitation,cloud_cover,wind_speed
0,2023-01-01T00:00,6.9,91,0.0,53,20.9
1,2023-01-01T01:00,6.9,92,0.0,32,20.7
2,2023-01-01T02:00,8.1,89,0.0,8,23.5
3,2023-01-01T03:00,7.6,91,0.0,6,22.1
4,2023-01-01T04:00,7.3,92,0.0,24,21.4
5,2023-01-01T05:00,7.4,93,0.0,45,22.4
6,2023-01-01T06:00,7.4,95,0.0,100,23.0
7,2023-01-01T07:00,7.5,95,0.0,100,22.5
8,2023-01-01T08:00,7.4,95,0.0,100,22.0
9,2023-01-01T09:00,7.4,95,0.0,100,22.1


In [183]:
print(df_weather.shape)
print(df.shape)

(12312, 6)
(12311, 2)


In [184]:
df['date'] = pd.to_datetime(df['date'])
df.dtypes

date     datetime64[ns]
price           float64
dtype: object

In [185]:
df_weather['date'] = pd.to_datetime(df_weather['date'])
df_weather.dtypes

date             datetime64[ns]
temperature             float64
humidity                  int64
precipitation           float64
cloud_cover               int64
wind_speed              float64
dtype: object

In [186]:
df.isnull().sum()

date     0
price    0
dtype: int64

In [187]:
df_weather.isnull().sum()

date             0
temperature      0
humidity         0
precipitation    0
cloud_cover      0
wind_speed       0
dtype: int64

In [188]:
df_merged = pd.merge(df, df_weather, on='date')
# df_merged.drop('date', axis=1, inplace=True)
df_merged.head()

,date,price,temperature,humidity,precipitation,cloud_cover,wind_speed
0,2023-01-01 00:00:00,13.31,6.9,91,0.0,53,20.9
1,2023-01-01 01:00:00,-0.07,6.9,92,0.0,32,20.7
2,2023-01-01 02:00:00,-0.47,8.1,89,0.0,8,23.5
3,2023-01-01 03:00:00,-1.56,7.6,91,0.0,6,22.1
4,2023-01-01 04:00:00,-1.94,7.3,92,0.0,24,21.4


In [189]:
df_merged.to_csv('../data/processed/price_data.csv', index=False)